In [28]:
using FilePaths; using FilePathsBase: /
using Plots
using TiffImages
using Images

LoadError: ArgumentError: Package Images not found in current path.
- Run `import Pkg; Pkg.add("Images")` to install the Images package.

In [19]:
dax_file = Path("/mnt/ibm_lg/merscope/merfish_raw_data/202205191322_MsKidney-wtK3-VZG116b-VT-061922_VMSC02401/data/stack_0_093.dax");
inf_file = Path("/") / Path(dax_file[1:end-1]) / splitext(dax_file[end])[1] * ".inf"

p"/mnt/ibm_lg/merscope/merfish_raw_data/202205191322_MsKidney-wtK3-VZG116b-VT-061922_VMSC02401/data/stack_0_093.inf"

In [20]:
size_re = r"frame dimensions = ([\d]+) x ([\d]+)";
length_re = r"number of frames = ([\d]+)";
endian_re = r" (big|little) endian";

In [21]:
inf_data = Dict()

for line in eachline(string(inf_file)) 
  if occursin(size_re, line)
    inf_data["image_width"], inf_data["image_height"] = tryparse.(Int, match(size_re, line));
    inf_data["image_size"] = inf_data["image_width"] * inf_data["image_height"]
  elseif occursin(length_re, line);
    inf_data["n_frames"] = tryparse(Int, match(length_re, line)[1])
  elseif occursin(endian_re, line)
    endian = match(endian_re, line)[1];
    inf_data["big_endian"] = endian == "big" ? true : false;
  end 
end
inf_data

Dict{Any, Any} with 5 entries:
  "big_endian"   => false
  "image_width"  => 2048
  "image_height" => 2048
  "n_frames"     => 22
  "image_size"   => 4194304

In [25]:
frame_number = 8;
buffer = Vector{UInt16}(undef, inf_data["image_size"])
open(dax_file, "r") do f
  seek(f, frame_number * inf_data["image_size"] * 2);
  read!(f, buffer)
  if inf_data["big_endian"]
    bswap(buffer)
  end
end

frame = reshape(buffer, inf_data["image_width"], inf_data["image_height"]);

In [26]:
heatmap(frame, color = :greys)

In [27]:
grays = reinterpret(Gray{N0f8}, frame)
tiff = TiffImages.DenseTaggedImage(grays)

LoadError: UndefVarError: N0f8 not defined